In [1]:
# !pip install "bokeh==2.1.0" "Jinja2==3.0.3"

import matplotlib.pyplot as plt
import networkx as nx
from bokeh.io import output_notebook, show
from bokeh.models import Circle, MultiLine, Range1d
from bokeh.palettes import Blues8
from bokeh.plotting import figure, from_networkx
from bokeh.transform import linear_cmap

from rs_graph.data import load_rs_graph_upstream_deps_dataset

###############################################################################

# Bokeh output to notebook
output_notebook()

# Matplotlib setup
plt.figure(figsize=(8, 8))
plt.tight_layout()

# Load dataset sample
df = load_rs_graph_upstream_deps_dataset().sample(sample_size)

ImportError: cannot import name 'Markup' from 'jinja2' (/home/eva/miniforge-pypy3/envs/rs-graph/lib/python3.11/site-packages/jinja2/__init__.py)

In [ ]:
# Create graph
graph = nx.from_pandas_edgelist(df, "upstream_dep_name", "repo")

# Calculate degree and attach to node data
degrees = dict(nx.degree(graph))
nx.set_node_attributes(graph, name="degree", values=degrees)

# Adjust size by degree
number_to_adjust_by = 5
adjusted_node_size = {
    node: degree + number_to_adjust_by for node, degree in nx.degree(graph)
}
nx.set_node_attributes(graph, name="adjusted_node_size", values=adjusted_node_size)

# Create a plot — set dimensions, toolbar, and title
plot = figure(
    tooltips=[("Package", "@index"), ("Degree", "@degree")],
    tools="pan,wheel_zoom,save,reset",
    active_scroll="wheel_zoom",
    x_range=Range1d(-10.1, 10.1),
    y_range=Range1d(-10.1, 10.1),
)

# Create a network graph object with spring layout
network_graph = from_networkx(graph, nx.spring_layout, scale=10, center=(0, 0))

# Set node size and color
# Set node sizes and colors according to node degree
# (color as spectrum of color palette)
minimum_value_color = min(
    network_graph.node_renderer.data_source.data["adjusted_node_size"]
)
maximum_value_color = max(
    network_graph.node_renderer.data_source.data["adjusted_node_size"]
)
network_graph.node_renderer.glyph = Circle(
    size="adjusted_node_size",
    fill_color=linear_cmap(
        "adjusted_node_size", Blues8, minimum_value_color, maximum_value_color
    ),
)

# Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

# Add network graph to the plot
plot.renderers.append(network_graph)

# Render
show(plot)